# TODO: Fix jogos_num

# Palpite
Cartola FC tips.

## Goal
To develop a model to predict how many points will an athlete score on Cartola FC.

The model is planned to be deployd on a web app and shall only use information available on the official Cartola FC API and on any popular betting API.

## Data Sources
### Cartola FC
The historical Cartola FC dataset used on this model is available on [github.com/henriquepgomide/caRtola](https://github.com/henriquepgomide/caRtola/tree/master/data).

### Betting Lines
The historical betting lines for Brasileirão Série A is available on [football-data.co.uk/brazil.php](https://www.football-data.co.uk/brazil.php).

Many thanks to the maintainers from both projects on making the data available for free to the world.

## Time Interval
The official Cartola FC API changed its format on 2018. Datasets before this year do not attend the expected formats, and therefore, won't be used.

The seasons used to build the model will be:

* 38 2018 rounds (complete season)
* 38 2019 rounds (complete season)
* 18 2020 rounds (almost half season)

It means:

* 2.5 seaons
* 94 rounds
* 940 matches

It seems to be enough and there is no need of trying to format previous years' datasets.

## Imports
Please make sure you have all packages installed before starting using the notebook.

In [1]:
import datetime
import itertools
import os
import pytz

import numpy as np
import pandas as pd
import sklearn
import xgboost
import sklearn.ensemble
import sklearn.linear_model
import sklearn.model_selection
import sklearn.pipeline
import sklearn.preprocessing

## Data Wrangling
### Load Cartola FC data
Load data from 2018, 2019 and 2020 Cartola FC seasons.

In [2]:
def load_all_rounds(season):
    """ 
    Load all rounds data from a season to a pandas DataFrame.
    
    Also rename columns to a shorter name and introduce a year feature.
    """
    # Define file paths.
    file_path_list = [f"data//cartola//{season}//rodada-{i + 1}.csv" for i in range(38)]
    # Load datasets into a list.
    rounds = [
        pd.read_csv(file_path, index_col=0) for file_path in file_path_list if os.path.exists(file_path)
        ]
    # Concatenate the list.
    data = pd.concat(rounds, ignore_index=True)

    # Remove and repetive term from columns names.
    data.columns = [col.replace("atletas.", "") for col in data.columns]

    # Create year column.
    data["year"] = [season for _ in range(data.shape[0])]

    return data


def load_seasons(seasons):
    """ Load specified seasons into a same pandas DataFrame. """
    return pd.concat([load_all_rounds(year) for year in seasons], ignore_index=True)
    

cartola_data = load_seasons([2018, 2019, 2020])
cartola_data.head()

,nome,slug,apelido,foto,atleta_id,rodada_id,clube_id,posicao_id,status_id,pontos_num,...,FT,GS,CV,GC,PP,DP,year,jogos_num,PI,DS
0,Matheus Ferraz Pereira,matheus-ferraz,Matheus Ferraz,https://s.glbimg.com/es/sde/f/2018/03/17/6d461...,38632,1,AME,zag,Nulo,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN
1,Willian Lanes de Lima,lima,Lima,https://s.glbimg.com/es/sde/f/2018/03/17/3d9ef...,38506,1,AME,zag,Nulo,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN
2,Rómulo Otero Vásquez,otero,Otero,https://s.glbimg.com/es/sde/f/2017/04/03/9fe40...,83004,1,ATL,mei,Provável,16.5,...,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN
3,Diego Ribas da Cunha,diego,Diego,https://s.glbimg.com/es/sde/f/2017/08/16/3ba37...,38909,1,FLA,mei,Provável,0.8,...,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN
4,Rodrigo Eduardo Costa Marinho,rodriguinho,Rodriguinho,https://s.glbimg.com/es/sde/f/2018/03/20/c125f...,61033,1,COR,mei,Provável,16.5,...,NaN,NaN,NaN,NaN,NaN,NaN,2018,NaN,NaN,NaN


### Number of Played Games
Most recent year come with a number o played gamed feature, however, 2018 doesn't have it. I'll recreate it for all years.

In [3]:
# Select scouts (stats) coumns.
scouts_cols = ['FC', 'FD', 'FF', 'FS', 'G', 'I', 'RB', 'CA', 'PE', 'A', 'SG', 'DD', 'FT', 'GS', 'CV', 'GC', 'PP', 'DP', 'PI', 'DS']
scouts = cartola_data[scouts_cols]

# If there are data on any of those columns, the player played the game.
cartola_data["played"] = scouts.notnull().any(axis=1).astype(int)

# Sort values, group by athlete and season and the calculate the cumulative sum of played games.
cartola_data = cartola_data.sort_values(["atleta_id", "year", "rodada_id"])
cartola_data["matches"] = cartola_data.groupby(["atleta_id", "year"]).cumsum()["played"]

### Keep relevant data
This model do not aim to use athletes scouts (stats). They are all removed. 

Redundant features, like athlete id, name, nickname, slug are also removed.

Price-related features are removed since I want the model to completely disregard prices (unbiased to pricing).

In [4]:
to_keep = [
    'year',  # Number of years played
    'apelido',  # Athlete nickname.
    'rodada_id',  # Round number.
    'clube.id.full.name',  # Club full name.
    'posicao_id',  # Position ID.
    'status_id',  # Status ID.
    'media_num',  # Mean number.
    'pontos_num',  # Points number.
    'matches',  # Number of matches played.
]

cartola_data = cartola_data[to_keep]
cartola_data.tail()

,year,apelido,rodada_id,clube.id.full.name,posicao_id,status_id,media_num,pontos_num,matches
74568,2020,Igor Torres,18,356,ata,Nulo,0.0,0.0,0
73816,2020,Sandrinho,17,290,ata,Nulo,0.0,0.0,0
74576,2020,Sandrinho,18,290,ata,Nulo,-3.0,-3.0,1
74577,2020,Ângelo,18,Santos,ata,Nulo,0.8,0.8,1
74575,2020,Brayan,18,Santos,ata,Nulo,0.0,0.0,0


### Translate
Code should preferable be written in English. 


Even though it is a brazilian fantasy game, with a brazilian target audience, 
you never know when someone else may be interested in reading your code for whatever the purpose it may be. 
So it is a nice idea to always keep it in English.

In [5]:
# Rename columns.
cartola_data = cartola_data.rename({
    "apelido": "name",
    "rodada_id": "round",
    "clube.id.full.name": "club",
    "posicao_id": "position",
    "status_id": "status",
    "pontos_num": "points",
    "media_num": "mean",
}, axis=1)

# Rename values from the column position.
cartola_data["position"] = cartola_data["position"].replace({
    "zag": "defender",
    "mei": "midfielder",
    "ata": "forward",
    "gol": "goalkeeper",
    "lat": "fullback",
    "tec": "coach",
})

# Rename values from the column status.
cartola_data["status"] = cartola_data["status"].replace({
    "Nulo": "null",
    "Provável": "expected",
    "Contundido": "injured",
    "Dúvida": "doubt",
    "Suspenso": "suspended",
})

cartola_data.tail()

,year,name,round,club,position,status,mean,points,matches
74568,2020,Igor Torres,18,356,forward,null,0.0,0.0,0
73816,2020,Sandrinho,17,290,forward,null,0.0,0.0,0
74576,2020,Sandrinho,18,290,forward,null,-3.0,-3.0,1
74577,2020,Ângelo,18,Santos,forward,null,0.8,0.8,1
74575,2020,Brayan,18,Santos,forward,null,0.0,0.0,0


### Clubs
It looks like there is pertubation in the `club` column.

Let's further invertigate it and decide how to fix it.

In [6]:
cartola_data["club"].unique()

array(['Flamengo', 'Avaí', 'Fluminense', 'Botafogo', '263', '293',
       'Bahia', 'Chapecoense', 'Ceará', '354', 'Goiás', 'Cruzeiro',
       'Palmeiras', '265', 'CSA', 'Atlético-PR', 'Athlético-PR', 'Sport',
       'Santos', 'Vasco', 'América-MG', '290', 'Grêmio', 'Atlético-MG',
       'Vitória', 'Fortaleza', '356', 'Internacional', 'Paraná',
       'Corinthians', 'Bragantino', '280', '264', '266', '284', '282',
       '292', 'Coritiba', '267', '275', 'São Paulo', '276', 'Atlético-GO',
       '373', '285'], dtype=object)

Some clubs names are represeted by its full name, while others are using the ID. To be coherent, I'll convert those IDs into names.

Since the names are case sensitive and have special characters that may produce errors later on, why not use IDs only? 
It is because the ID system changed throughout the years. It means that a same club may have different IDs for the year 2018 and 2019. It could produce erros that are harder to debug.

Even though full names are also prone to error, they are easier to debug, since they are written in natural language.

In order to correlate IDs to names, first it is important to understant in which year it is happening to use the most appropiate mapping.


In [7]:

def is_int(value):
    """ Check if a value is integer. """
    try:
        int(value)
        return True
    except ValueError:
        return False


def count_int(sequence):
    """ Count the amount of integers in a sequence. """
    return len([value for value in sequence if is_int(value)])


print(f"There are {count_int(cartola_data[cartola_data['year'] == 2018]['club'])} IDs in the year 2018")
print(f"There are {count_int(cartola_data[cartola_data['year'] == 2019]['club'])} IDs in the year 2019")
print(f"There are {count_int(cartola_data[cartola_data['year'] == 2020]['club'])} IDs in the year 2020")

There are 0 IDs in the year 2018
There are 0 IDs in the year 2019
There are 4396 IDs in the year 2020


All of the IDs are from the 2020 year. So I'll use the mapping from this year. The clubs dataset was retrieved from the official Cartola FC API.

In [8]:
clubs_data = pd.read_csv(r"data\cartola\clubes.csv", index_col=0)
clubs_data.head()

,id,nome,abreviacao,escudos,nome_fantasia
1,1,Outros,OUT,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Outros
1349,1349,Ipatinga,IPA,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Ipatinga
1371,1371,Cuiabá,CUI,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Cuiabá
1390,1390,Icasa,ICA,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Icasa
2190,2190,Oeste,OES,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Oeste


#### Translate Clubs Dataset
For the same reason as before, the dataset needs to be translated.

In [9]:
# Rename columns.
clubs_data = clubs_data.rename({
    "nome": "name",
    "abreviacao": "abbreviation",
    "escudos": "logos",
    "nome_fantasia": "fanstasy_name",
}, axis=1)

clubs_data.head()

,id,name,abbreviation,logos,fanstasy_name
1,1,Outros,OUT,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Outros
1349,1349,Ipatinga,IPA,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Ipatinga
1371,1371,Cuiabá,CUI,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Cuiabá
1390,1390,Icasa,ICA,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Icasa
2190,2190,Oeste,OES,{'60x60': 'https://s.glbimg.com/es/sde/f/organ...,Oeste


In [10]:
def apply_mapping(value, mapping):
    """ Apply a mapping to column values. """
    if value in mapping:
        return mapping[value]
    return value



id_to_name_mapping = {str(row.id): row.name for row in clubs_data.itertuples()}

cartola_data["club"] = cartola_data["club"].apply(apply_mapping, mapping=id_to_name_mapping)
cartola_data["club"].unique()

array(['Flamengo', 'Avaí', 'Fluminense', 'Botafogo', 'Athlético-PR',
       'Bahia', 'Chapecoense', 'Ceará', 'Goiás', 'Cruzeiro', 'Palmeiras',
       'CSA', 'Atlético-PR', 'Sport', 'Santos', 'Vasco', 'América-MG',
       'Grêmio', 'Atlético-MG', 'Vitória', 'Fortaleza', 'Internacional',
       'Paraná', 'Corinthians', 'Bragantino', 'Coritiba', 'São Paulo',
       'Atlético-GO'], dtype=object)

We are almost done with the clubs. The last thing that needs to be fixed is the name from Athlético-PR.

This team changes its name in recent years and because of it, is duplicated.

In [11]:
cartola_data["club"] = cartola_data["club"].replace("Atlético-PR", "Athlético-PR")
cartola_data["club"].unique()

array(['Flamengo', 'Avaí', 'Fluminense', 'Botafogo', 'Athlético-PR',
       'Bahia', 'Chapecoense', 'Ceará', 'Goiás', 'Cruzeiro', 'Palmeiras',
       'CSA', 'Sport', 'Santos', 'Vasco', 'América-MG', 'Grêmio',
       'Atlético-MG', 'Vitória', 'Fortaleza', 'Internacional', 'Paraná',
       'Corinthians', 'Bragantino', 'Coritiba', 'São Paulo',
       'Atlético-GO'], dtype=object)

### Data Transformation
There is still one big problem.

The following columns vary from game to game. 

* `status`
* `points`
* `mean`
* `matches`

Status is available before the match to the user, while mean is available after the match. In the next round the user will have status for the next match, and the mean from the last match.

Therefore, for a model to be useful for the user, it should consider next match status, and last match mean.

Naturally, for the first match, mean will be null. Also, Notice that points won't be transformed since it is the target feature.

In [12]:
# Duplicate data and reduce the round feature
antecipated_data = cartola_data.copy()
antecipated_data["round"] = [round_ - 1 for round_ in antecipated_data["round"]]

# Join on features that don't change
join_on = ['year', 'name', 'round', 'club', 'position']
cartola_data = pd.merge(
    left=antecipated_data, 
    right=cartola_data, 
    on=join_on, 
    how="left", 
    suffixes=("_current", "_previous")  # Translate to _current, _previous
    )

# Reset rounds.
cartola_data["round"] = [round + 1 for round in cartola_data["round"]]

# Remove columns that won't be no longer used.
cartola_data = cartola_data.drop(columns=["status_previous", "points_previous", "mean_current"])

# Remove suffixes
cartola_data.columns = [col.replace("_previous", "").replace("_current", "") for col in cartola_data.columns]

# Transform mean and matches NaN in 0
cartola_data["mean"] = cartola_data["mean"].fillna(0)
cartola_data["matches"] = cartola_data["matches"].fillna(0)

cartola_data.head()

,year,name,round,club,position,status,points,matches,mean,matches
0,2018,Juan,1,Flamengo,defender,expected,0.5,1.0,0.00,0.0
1,2018,Juan,2,Flamengo,defender,doubt,0.0,2.0,0.50,1.0
2,2018,Juan,3,Flamengo,defender,expected,5.0,3.0,0.50,2.0
3,2018,Juan,4,Flamengo,defender,expected,0.0,4.0,2.75,3.0
4,2018,Juan,5,Flamengo,defender,injured,2.3,5.0,2.75,4.0


### Dates
We should introduce a dates feature. This is important in order to merge with the betting lines dataset later on.

I'll concatenate the matches datasets from each year.

In [13]:
def load_date_dataset(year):
    """ Load dates dataset. """
    data = pd.read_csv(f"data\\cartola\\{year}\\{year}_partidas.csv")
    data["year"] = [year for _ in range(data.shape[0])]
    return data


# Load dates datasets.
years = [2018, 2019, 2020]
matches_data = pd.concat([load_date_dataset(year) for year in years], ignore_index=True)

print(f"There are {count_int(matches_data[matches_data['year'] == 2018]['home_team'])} IDs in the year 2018")
print(f"There are {count_int(matches_data[matches_data['year'] == 2019]['home_team'])} IDs in the year 2019")
print(f"There are {count_int(matches_data[matches_data['year'] == 2020]['home_team'])} IDs in the year 2020")

matches_data.head()

There are 0 IDs in the year 2018
There are 380 IDs in the year 2019
There are 180 IDs in the year 2020


,game,round,date,home_team,score,away_team,arena,year,home_score,away_score
0,1.0,1,14/04/2018 - 16:00,Cruzeiro - MG,0 x 1,Grêmio - RS,Mineirão - Belo Horizonte - MG,2018,NaN,NaN
1,2.0,1,15/04/2018 - 19:00,Atlético - PR,5 x 1,Chapecoense - SC,Arena da Baixada - Curitiba - PR,2018,NaN,NaN
2,3.0,1,15/04/2018 - 11:00,América - MG,3 x 0,Sport - PE,Independência - Belo Horizonte - MG,2018,NaN,NaN
3,4.0,1,14/04/2018 - 19:00,Vitória - BA,2 x 2,Flamengo - RJ,Manoel Barradas - Salvador - BA,2018,NaN,NaN
4,5.0,1,15/04/2018 - 16:00,Vasco da Gama - RJ,2 x 1,Atlético - MG,São Januário - Rio de Janeiro - RJ,2018,NaN,NaN


This dataset also have some problems that should be fixed:

* Dates are in different format depending on the year.
* Team names are in various formats.

To fix teams names I'll use the `time_ids.csv` dataset. It as some other teams names mapping.


In [14]:
alternative_clubs_data = pd.read_csv(r"data\cartola\times_ids.csv")
alternative_clubs_data.head()

,nome.cbf,nome.cartola,nome.completo,cod.older,cod.2017,cod.2018,id,abreviacao,escudos.60x60,escudos.45x45,escudos.30x30
0,América - MG,América-MG,America MG,327,327,327,327,AME,https://s.glbimg.com/es/sde/f/organizacoes/201...,https://s.glbimg.com/es/sde/f/organizacoes/201...,https://s.glbimg.com/es/sde/f/organizacoes/201...
1,America - RN,Atlético-RN,America RN,200,200,1,200,OUT,NaN,NaN,NaN
2,Atlético - GO,Atlético-GO,Atletico GO,201,373,373,373,ATL,NaN,NaN,NaN
3,Atlético - MG,Atlético-MG,Atletico Mineiro,282,282,282,282,ATL,https://s.glbimg.com/es/sde/f/equipes/2017/11/...,https://s.glbimg.com/es/sde/f/equipes/2017/11/...,https://s.glbimg.com/es/sde/f/equipes/2017/11/...
4,Atlético - PR,Atlético-PR,Atletico Paranaense,293,293,293,293,ATL,https://s.glbimg.com/es/sde/f/equipes/2015/06/...,https://s.glbimg.com/es/sde/f/equipes/2015/06/...,https://s.glbimg.com/es/sde/f/equipes/2015/06/...


In [15]:
# Translate
alternative_clubs_data = alternative_clubs_data.rename({
    "nome.cbf": "cbf_name",  # CBF is the brazilian soccer confederation.
    "nome.cartola": "cartola_name",  # Cartola FC name.
    "nome.completo": "full_name",
    "cod_older": "id_old",  # ID as before 2017.
    "cod.2017": "id_2017",  # ID as 2017
    "cod.2018": "id_2018",  # ID as 2018
    "id": "id_2019",  # ID as 2019
    "abreviacao": "abbreviation",
    "escudo.60x60": "logo_60x60",
    "escudo.45x45": "logo_45x45",
    "escudo.30x30": "logo_30x30",
}, axis=1)

alternative_clubs_data["cartola_name"] = alternative_clubs_data["cartola_name"].replace(
    "Atlético-PR", "Athlético-PR"
    )

alternative_clubs_data.head()

,cbf_name,cartola_name,full_name,cod.older,id_2017,id_2018,id_2019,abbreviation,escudos.60x60,escudos.45x45,escudos.30x30
0,América - MG,América-MG,America MG,327,327,327,327,AME,https://s.glbimg.com/es/sde/f/organizacoes/201...,https://s.glbimg.com/es/sde/f/organizacoes/201...,https://s.glbimg.com/es/sde/f/organizacoes/201...
1,America - RN,Atlético-RN,America RN,200,200,1,200,OUT,NaN,NaN,NaN
2,Atlético - GO,Atlético-GO,Atletico GO,201,373,373,373,ATL,NaN,NaN,NaN
3,Atlético - MG,Atlético-MG,Atletico Mineiro,282,282,282,282,ATL,https://s.glbimg.com/es/sde/f/equipes/2017/11/...,https://s.glbimg.com/es/sde/f/equipes/2017/11/...,https://s.glbimg.com/es/sde/f/equipes/2017/11/...
4,Atlético - PR,Athlético-PR,Atletico Paranaense,293,293,293,293,ATL,https://s.glbimg.com/es/sde/f/equipes/2015/06/...,https://s.glbimg.com/es/sde/f/equipes/2015/06/...,https://s.glbimg.com/es/sde/f/equipes/2015/06/...


In [16]:
# Create a dictionary from the dataset.
cbf_to_cartola_mapping = {
    row.cbf_name: row.cartola_name for row in alternative_clubs_data.itertuples()
    }
id_to_name_mapping_2019 = {
    row.id_2019: row.cartola_name for row in alternative_clubs_data.itertuples()
    }
id_to_name_mapping_2020 = {int(row.id): row.name for row in clubs_data.itertuples()}

# Apply CBF name mapping function.
year_to_mapping = (
    ([2018, 2019, 2020], cbf_to_cartola_mapping,),
    ([2019], id_to_name_mapping_2019,),
    ([2020], id_to_name_mapping_2020,),
)

for years, mapping in year_to_mapping:
    # Filter.
    matches_data_year = matches_data[matches_data["year"].isin(years)]
    
    # Map.
    matches_data_year["home_team"] = matches_data_year["home_team"].apply(
        apply_mapping, mapping=mapping
        )
    matches_data_year["away_team"] = matches_data_year["away_team"].apply(
        apply_mapping, mapping=mapping
        )

    # Apply.
    matches_data[matches_data["year"].isin(years)] = matches_data_year

matches_data["home_team"].unique()

array(['Cruzeiro', 'Athlético-PR', 'América-MG', 'Vitória', 'Vasco',
       'Botafogo', 'São Paulo', 'Santos', 'Corinthians', 'Internacional',
       'Atlético-MG', 'Paraná', 'Chapecoense', 'Bahia', 'Fluminense',
       'Flamengo', 'Ceará-SC', 'Sport', 'Palmeiras', 'Grêmio', 'Avaí',
       'CSA', 'Fortaleza', 'Goiás', 'Coritiba', 'Bragantino', 'Ceará',
       'Atlético-GO'], dtype=object)

Ceará appears twice as *Ceará-SC* and *Ceará*. It should be fixed.

In [17]:
matches_data["home_team"] = matches_data["home_team"].replace("Ceará-SC", "Ceará")
matches_data["away_team"] = matches_data["away_team"].replace("Ceará-SC", "Ceará")

matches_data["home_team"].unique()

array(['Cruzeiro', 'Athlético-PR', 'América-MG', 'Vitória', 'Vasco',
       'Botafogo', 'São Paulo', 'Santos', 'Corinthians', 'Internacional',
       'Atlético-MG', 'Paraná', 'Chapecoense', 'Bahia', 'Fluminense',
       'Flamengo', 'Ceará', 'Sport', 'Palmeiras', 'Grêmio', 'Avaí', 'CSA',
       'Fortaleza', 'Goiás', 'Coritiba', 'Bragantino', 'Atlético-GO'],
      dtype=object)

We're done with teams, now is time to clean up dates.

In [18]:
def interpret_date(string, patterns, timezone=None):
    """ Interpret some specific patterns of date. """
    for pattern in patterns:
        try:
            dt = datetime.datetime.strptime(string, pattern)
            if timezone:
                dt = dt + datetime.timedelta(hours=timezone)
            return pd.to_datetime(dt.date())
        except ValueError:
            pass
    raise ValueError("Couldn't match any date pattern.")


# Interpret string ad datetime.
matches_data["date"] = matches_data["date"].apply(
    interpret_date, patterns=["%d/%m/%Y - %H:%M", "%Y-%m-%d"])

matches_data.head(5)

,game,round,date,home_team,score,away_team,arena,year,home_score,away_score
0,1.0,1,2018-04-14,Cruzeiro,0 x 1,Grêmio,Mineirão - Belo Horizonte - MG,2018,NaN,NaN
1,2.0,1,2018-04-15,Athlético-PR,5 x 1,Chapecoense,Arena da Baixada - Curitiba - PR,2018,NaN,NaN
2,3.0,1,2018-04-15,América-MG,3 x 0,Sport,Independência - Belo Horizonte - MG,2018,NaN,NaN
3,4.0,1,2018-04-14,Vitória,2 x 2,Flamengo,Manoel Barradas - Salvador - BA,2018,NaN,NaN
4,5.0,1,2018-04-15,Vasco,2 x 1,Atlético-MG,São Januário - Rio de Janeiro - RJ,2018,NaN,NaN


### Merge Cartola FC and matches datasets
Now that everything looks fine with the matches dataset, it is ready to be merged on the Cartola FC dataset.

In [19]:
# Rename the matches dataset to attend the cartola dataset features names. This is required for merging.
matches_renamed = matches_data.rename({
    "home_team": "club_home",
    "away_team": "club_away"
    }, axis=1)

# We need to do it in two steps. First merge searching for the club in home teams. 
# Then do it again for away teams.

for club_type in ["home", "away"]:
    matches_renamed["club"] = matches_renamed[f"club_{club_type}"]

    # Filter to merge only desired features.
    matches_renamed_filtered = matches_renamed[["round", "year", "club", "date"]]

    merge_on = ["round", "year", "club"]
    cartola_data = pd.merge(cartola_data, matches_renamed_filtered, on=merge_on, how="left")

# Unify both new columns in a single column.
cartola_data["date"] = np.where(
    pd.isna(cartola_data["date_x"]),
    cartola_data["date_y"],
    cartola_data["date_x"]
    )

# Remove date_x and date_y.
cartola_data = cartola_data.drop(columns=["date_x", "date_y"])

cartola_data.tail()

,year,name,round,club,position,status,points,matches,mean,matches,date
74757,2020,Igor Torres,18,Fortaleza,forward,null,0.0,0.0,0.0,0.0,2020-10-24
74758,2020,Sandrinho,17,Goiás,forward,null,0.0,0.0,0.0,0.0,2020-10-19
74759,2020,Sandrinho,18,Goiás,forward,null,-3.0,1.0,0.0,0.0,2020-10-24
74760,2020,Ângelo,18,Santos,forward,null,0.8,1.0,0.0,0.0,2020-10-25
74761,2020,Brayan,18,Santos,forward,null,0.0,0.0,0.0,0.0,2020-10-25


Check if all columns received a date value.

In [20]:
# Count NaNs in each row.
cartola_data.shape[0] - cartola_data.count()

year          0
name          0
round         0
club          0
position      0
status        0
points        0
matches       0
mean          0
matches       0
date        458
dtype: int64

There are still 458 rows without date. They needed to be investigated.

In [21]:
cartola_data[pd.isna(cartola_data["date"])].groupby(["year", "round", "club"]).first()

name    position     status  points  \
year round club                                                          
2020 1     Atlético-GO   Vagner Mancini       coach   expected     0.0   
           Bahia               Anderson  goalkeeper       null     0.0   
           Botafogo       Paulo Autuori       coach   expected     0.0   
           Corinthians               Jô     forward   expected     0.0   
     11    Athlético-PR        Jonathan    fullback   expected     0.0   
           Bahia           Mano Menezes       coach   expected     0.0   
           Flamengo         Diego Alves  goalkeeper    injured     0.0   
           Goiás           Rafael Moura     forward   expected     0.0   
           São Paulo           Hernanes  midfielder      doubt     0.0   
     12    Sport           Thiago Neves  midfielder   expected     0.0   
     18    Corinthians               Jô     forward       null     0.0   
           Vasco         Leandro Castan    defender  suspended     0.0   

                         matches  mean  matches date  
year round club                                       
2020 1     Atlético-GO       0.0  0.00      0.0  NaT  
           Bahia             0.0  0.00      0.0  NaT  
           Botafogo          0.0  0.00      0.0  NaT  
           Corinthians       0.0  0.00      0.0  NaT  
     11    Athlético-PR     11.0  2.72     10.0  NaT  
           Bahia             0.0  0.77      0.0  NaT  
           Flamengo         11.0  2.22     10.0  NaT  
           Goiás             6.0  0.00      0.0  NaT  
           São Paulo         6.0  3.14      5.0  NaT  
     12    Sport             8.0  0.10      7.0  NaT  
     18    Corinthians      17.0  2.51     16.0  NaT  
           Vasco            17.0  3.22     16.0  NaT

It looks likes those are games that really didn't happen. 2020 was a year with a lot of delayed games due to the COVID-19 pandemic.

In [22]:
cartola_data = cartola_data.dropna()

# Count NaNs in each row.
cartola_data.shape[0] - cartola_data.count()

year        0
name        0
round       0
club        0
position    0
status      0
points      0
matches     0
mean        0
matches     0
date        0
dtype: int64

### Load betting lines data
Load betting lines from 2018, 2019 and 2020 seasons.

In [23]:
bet_data = pd.read_csv(r"data\betting\historical_betting_lines.csv")
bet_data.head()

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
0,Brazil,Serie A,2012,19/05/2012,22:30,Palmeiras,Portuguesa,1.0,1.0,D,1.75,3.86,5.25,1.76,3.87,5.31,1.69,3.50,4.90
1,Brazil,Serie A,2012,19/05/2012,22:30,Sport Recife,Flamengo RJ,1.0,1.0,D,2.83,3.39,2.68,2.83,3.42,2.70,2.59,3.23,2.58
2,Brazil,Serie A,2012,20/05/2012,01:00,Figueirense,Nautico,2.0,1.0,H,1.60,4.04,6.72,1.67,4.05,7.22,1.59,3.67,5.64
3,Brazil,Serie A,2012,20/05/2012,20:00,Botafogo RJ,Sao Paulo,4.0,2.0,H,2.49,3.35,3.15,2.49,3.39,3.15,2.35,3.26,2.84
4,Brazil,Serie A,2012,20/05/2012,20:00,Corinthians,Fluminense,0.0,1.0,A,1.96,3.53,4.41,1.96,3.53,4.41,1.89,3.33,3.89


In [24]:
# Filter seasons.
bet_data = bet_data[bet_data["Season"].isin([2018, 2019, 2020])]
bet_data.head()

,Country,League,Season,Date,Time,Home,Away,HG,AG,Res,PH,PD,PA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA
2280,Brazil,Serie A,2018,14/04/2018,20:00,Cruzeiro,Gremio,0.0,1.0,A,1.96,3.29,4.87,2.11,3.30,4.87,1.98,3.16,4.15
2281,Brazil,Serie A,2018,14/04/2018,23:00,Vitoria,Flamengo RJ,2.0,2.0,D,3.56,3.41,2.22,3.75,3.42,2.24,3.40,3.31,2.14
2282,Brazil,Serie A,2018,15/04/2018,01:00,Santos,Ceara,2.0,0.0,H,1.57,4.13,6.76,1.60,4.14,7.10,1.55,3.88,6.12
2283,Brazil,Serie A,2018,15/04/2018,15:00,America MG,Sport Recife,3.0,0.0,H,1.98,3.35,4.51,2.05,3.42,4.51,1.96,3.27,4.01
2284,Brazil,Serie A,2018,15/04/2018,20:00,Corinthians,Fluminense,2.0,1.0,H,1.67,3.68,6.55,1.68,3.74,7.00,1.62,3.55,6.03


### Keep relevant rows
We are only interested in information that is available before the game.

We take as premise that the betting odds are accurate, and we shall not try to predict whenever they will fail. This would make the model over-complicated.

With that in mind, we only need the odds and the minimum information required to merge them with the Cartola FC dataset.

#### About the odds
We will use average odds instead of maximum or from a single source to avoid outliers or bad decisions by a specific booker.

In [25]:
to_keep = ["Season", "Date", "Time", "Home", "Away", "AvgH", "AvgD", "AvgA"]
bet_data = bet_data[to_keep]
bet_data.head()

,Season,Date,Time,Home,Away,AvgH,AvgD,AvgA
2280,2018,14/04/2018,20:00,Cruzeiro,Gremio,1.98,3.16,4.15
2281,2018,14/04/2018,23:00,Vitoria,Flamengo RJ,3.40,3.31,2.14
2282,2018,15/04/2018,01:00,Santos,Ceara,1.55,3.88,6.12
2283,2018,15/04/2018,15:00,America MG,Sport Recife,1.96,3.27,4.01
2284,2018,15/04/2018,20:00,Corinthians,Fluminense,1.62,3.55,6.03


### Fit data format into Cartola FC dataset standards

Everything is in a different format related to the Cartola FC dataset. For a succesful merge this dataset needs to be reformatted.

In [26]:
# Concatenate Date and Time
bet_data["Date"] = [f"{row.Date} {row.Time}" for row in bet_data.itertuples()]
bet_data = bet_data.drop(columns=["Time"])

# Rename columns.
bet_data = bet_data.rename({
    "Season": "year",
    "Date": "date",
    "Home": "home",
    "Away": "away",
    "AvgH": "home_odds",
    "AvgD": "draw_odds",
    "AvgA": "away_odds",
}, axis=1)

# Interpret string ad datetime.
bet_data["date"] = bet_data["date"].apply(
    interpret_date, patterns=["%d/%m/%Y %H:%M"], timezone=-4,
    )

bet_data.head()

,year,date,home,away,home_odds,draw_odds,away_odds
2280,2018,2018-04-14,Cruzeiro,Gremio,1.98,3.16,4.15
2281,2018,2018-04-14,Vitoria,Flamengo RJ,3.40,3.31,2.14
2282,2018,2018-04-14,Santos,Ceara,1.55,3.88,6.12
2283,2018,2018-04-15,America MG,Sport Recife,1.96,3.27,4.01
2284,2018,2018-04-15,Corinthians,Fluminense,1.62,3.55,6.03


And now for something completely different... fix clubs names.

This time is more challenging because it doesn't seem to match any pattern used before. I will have to carefully create a new mapping.

In [27]:
bet_data["home"].unique()

array(['Cruzeiro', 'Vitoria', 'Santos', 'America MG', 'Corinthians',
       'Internacional', 'Vasco', 'Atletico-PR', 'Botafogo RJ',
       'Sao Paulo', 'Bahia', 'Flamengo RJ', 'Parana', 'Atletico-MG',
       'Ceara', 'Chapecoense-SC', 'Fluminense', 'Palmeiras', 'Gremio',
       'Sport Recife', 'Athletico-PR', 'CSA', 'Avai', 'Fortaleza',
       'Goias', 'Coritiba', 'Bragantino', 'Atletico GO'], dtype=object)

To be corrected:

* No accentuation. 
* Some teams like Botafogo and Flamengo have the states or city appended to end.
* Some teams uses dashs and other spaces.

In [28]:
teams_names_mapping = {
    "Vitoria": "Vitória",
    "America MG": "América-MG",
    "Sao Paulo": "São Paulo",
    "Flamengo RJ": "Flamengo",
    "Botafogo RJ": "Botafogo",
    "Parana": "Paraná",
    "Atletico-MG": "Atlético-MG",
    "Ceara": "Ceará",
    "Chapecoense-SC": "Chapecoense",
    "Gremio": "Grêmio",
    "Sport Recife": "Sport",
    "Athletico-PR": "Athlético-PR",
    "Atletico-PR": "Athlético-PR",
    "Avai": "Avaí",
    "Goias": "Goiás",
    "Atletico GO": "Atlético-GO",

}

# Apply mapping.
for col in ["home", "away"]:
    bet_data[col] = bet_data[col].apply(apply_mapping, mapping=teams_names_mapping)

    # Check that new columns names are the same from Cartola FC dataset.
    if all([team in cartola_data["club"].unique() for team in bet_data[col].unique()]):
        print(f"Formatting was succesfull on column {col}.")
    else:
        print(f"Something wrong with column {col} formatting.")

bet_data.head()

Formatting was succesfull on column home.
Formatting was succesfull on column away.


,year,date,home,away,home_odds,draw_odds,away_odds
2280,2018,2018-04-14,Cruzeiro,Grêmio,1.98,3.16,4.15
2281,2018,2018-04-14,Vitória,Flamengo,3.40,3.31,2.14
2282,2018,2018-04-14,Santos,Ceará,1.55,3.88,6.12
2283,2018,2018-04-15,América-MG,Sport,1.96,3.27,4.01
2284,2018,2018-04-15,Corinthians,Fluminense,1.62,3.55,6.03


## Merge Both Datasets
Finally the time has come to merge both datasets. 

In [29]:
# There may be problems with timezone, so consider an one day tolerance.
tol = pd.Timedelta(value=1, unit="day")
# merge_asof requires sorting the left variable.
cartola_data = cartola_data.sort_values("date", ignore_index=True)

# First merge home games.
bet_data["club"] = bet_data["home"]
home_games = pd.merge_asof(cartola_data, bet_data, on="date", by="club", direction="nearest", tolerance=tol)#, how="left")

# First merge away games.
bet_data["club"] = bet_data["away"]
away_games = pd.merge_asof(cartola_data, bet_data, on="date", by="club", direction="nearest", tolerance=tol)#, how="left")

cartola_data["win"] = np.where(
    pd.isna(home_games["home_odds"]),
    away_games["away_odds"],
    home_games["home_odds"]
    )

cartola_data["lose"] = np.where(
    pd.isna(home_games["away_odds"]),
    away_games["home_odds"],
    home_games["away_odds"]
)

cartola_data["draw"] = np.where(
    pd.isna(home_games["draw_odds"]),
    away_games["draw_odds"],
    home_games["draw_odds"]
)

cartola_data.head()

,year,name,round,club,position,status,points,matches,mean,matches,date,win,lose,draw
0,2018,Juan,1,Flamengo,defender,expected,0.5,1.0,0.0,0.0,2018-04-14,2.14,3.40,3.31
1,2018,Yago,1,Vitória,midfielder,expected,7.5,1.0,0.0,0.0,2018-04-14,3.40,2.14,3.31
2,2018,Diego Pituca,1,Santos,midfielder,null,0.0,0.0,0.0,0.0,2018-04-14,1.55,6.12,3.88
3,2018,Wescley,1,Ceará,midfielder,expected,0.9,1.0,0.0,0.0,2018-04-14,6.12,1.55,3.88
4,2018,Jonatas Belusso,1,Vitória,forward,null,0.0,0.0,0.0,0.0,2018-04-14,3.40,2.14,3.31


### Check NaNs

Let's check if the merger was success.

In [30]:
# NaNs ratio in each row.
(cartola_data.shape[0] - cartola_data.count()) / cartola_data.shape[0]

year        0.000000
name        0.000000
round       0.000000
club        0.000000
position    0.000000
status      0.000000
points      0.000000
matches     0.000000
mean        0.000000
matches     0.000000
date        0.000000
win         0.011143
lose        0.011143
draw        0.011143
dtype: float64

The merge was unsuccesful for around 1% of occurences. This is negligible. They can be dropped.

In [31]:
cartola_data = cartola_data.dropna()

### Check Merging
This merger is the most important action so far. It is crutial that it was done correctly, otherwise the model will be completely wrong. It is a good idea to check it.

In [32]:
# Extract a sample from Cartola FC dataframe.
sample = cartola_data.sample(1).iloc[0]

# Find the same game in the betting datase.
bet_sample = bet_data[
    (bet_data["date"] == sample["date"])
    & ((bet_data["home"] == sample["club"]) | (bet_data["away"] == sample["club"]))
].iloc[0]

# Check if drawing odds is the same.
is_odds_equal = sample["draw"] == bet_sample["draw_odds"]
print("Merge was succesfull for this sample.\n")

# Visualize both samples
print("Cartola FC dataset sample:\n", sample, "\n")
print("Betting dataset sample:\n", bet_sample)

Merge was succesfull for this sample.

Cartola FC dataset sample:
 year                       2018
name                    Emanuel
round                         9
club                     Grêmio
position               defender
status                     null
points                        0
matches                       0
mean                          0
matches                       0
date        2018-06-03 00:00:00
win                        2.55
lose                       2.98
draw                       2.94
Name: 6966, dtype: object 

Betting dataset sample:
 year                        2018
date         2018-06-03 00:00:00
home                       Bahia
away                      Grêmio
home_odds                   2.98
draw_odds                   2.94
away_odds                   2.55
club                      Grêmio
Name: 2364, dtype: object


It confirms that pd.merge_asof was correctly set.

### Transform Odds in Events Probability.
I want to disregard the pay-off and focus on probability on the odds. To do so, I'll transform odds in probabilities by normalizing them.

In [33]:
odds_sum = cartola_data[["win", "draw", "lose"]].sum(1)
cartola_data["win"] = cartola_data["win"] / odds_sum
cartola_data["lose"] = cartola_data["lose"] / odds_sum
cartola_data["draw"] = cartola_data["draw"] / odds_sum
cartola_data.head()

,year,name,round,club,position,status,points,matches,mean,matches,date,win,lose,draw
0,2018,Juan,1,Flamengo,defender,expected,0.5,1.0,0.0,0.0,2018-04-14,0.241808,0.384181,0.374011
1,2018,Yago,1,Vitória,midfielder,expected,7.5,1.0,0.0,0.0,2018-04-14,0.384181,0.241808,0.374011
2,2018,Diego Pituca,1,Santos,midfielder,null,0.0,0.0,0.0,0.0,2018-04-14,0.134199,0.529870,0.335931
3,2018,Wescley,1,Ceará,midfielder,expected,0.9,1.0,0.0,0.0,2018-04-14,0.529870,0.134199,0.335931
4,2018,Jonatas Belusso,1,Vitória,forward,null,0.0,0.0,0.0,0.0,2018-04-14,0.384181,0.241808,0.374011


### Keep Only Relevand Features
Some features like date and athlete name were usefull up to know only for merging datasets and debugging operations. However, they are no longer relevant, since those operations are completed.

Also, now we'll work with a single DataFrame and there's no need to call the variable `cartola_data`. I'll rename it to only `data`.

In [34]:
data = cartola_data[["position", "status", "matches", "mean", "win", "lose", "draw", "points"]]
data.head()

,position,status,matches,matches,mean,win,lose,draw,points
0,defender,expected,1.0,0.0,0.0,0.241808,0.384181,0.374011,0.5
1,midfielder,expected,1.0,0.0,0.0,0.384181,0.241808,0.374011,7.5
2,midfielder,null,0.0,0.0,0.0,0.134199,0.529870,0.335931,0.0
3,midfielder,expected,1.0,0.0,0.0,0.529870,0.134199,0.335931,0.9
4,forward,null,0.0,0.0,0.0,0.384181,0.241808,0.374011,0.0


### Keep Only Relevant Rows
If the player won't play, than there is no reason for considering him at all. It would only makes hurt our model.

In [35]:
data = data[
    (data["status"] != "suspended")
    & (data["status"] != "injured")
    & (data["status"] != "null")
]
data.head()

,position,status,matches,matches,mean,win,lose,draw,points
0,defender,expected,1.0,0.0,0.0,0.241808,0.384181,0.374011,0.5
1,midfielder,expected,1.0,0.0,0.0,0.384181,0.241808,0.374011,7.5
3,midfielder,expected,1.0,0.0,0.0,0.529870,0.134199,0.335931,0.9
6,midfielder,expected,1.0,0.0,0.0,0.241808,0.384181,0.374011,0.8
9,midfielder,expected,1.0,0.0,0.0,0.134199,0.529870,0.335931,6.5


### Features Encoding
`position` and `status` are the only non-numerical features. They nedded to be encoded before modelling.

#### Rrders
Orders are fundamental in order to create a meaningfull encoding. The following orders will be used:

* `position`: The most close to the attack a player plays, more likely it is to score more points. The order is based on the further to the attack (coach and then goalkeeper), to the closest (forward).
* `status`: The most minutes a player plays, the most likely it will score points. The order is based on the less likely to play (suspended) to the most likely to play (expected);

In [36]:
# Converto features to category-type.
data["position"] = data["position"].astype("category")
data["status"] = data["status"].astype("category")

# Order
position_order = ["coach", "goalkeeper", "defender", "fullback", "midfielder", "forward"]
status_order = ["doubt", "expected"]
data["position"] = data["position"].cat.reorder_categories(position_order)
data["status"] = data["status"].cat.reorder_categories(status_order)

# Transform into codes.
data["position"] = data["position"].cat.codes
data["status"] = data["status"].cat.codes

data.head()

,position,status,matches,matches,mean,win,lose,draw,points
0,2,1,1.0,0.0,0.0,0.241808,0.384181,0.374011,0.5
1,4,1,1.0,0.0,0.0,0.384181,0.241808,0.374011,7.5
3,4,1,1.0,0.0,0.0,0.529870,0.134199,0.335931,0.9
6,4,1,1.0,0.0,0.0,0.241808,0.384181,0.374011,0.8
9,4,1,1.0,0.0,0.0,0.134199,0.529870,0.335931,6.5


### Save Data
Save data and create a read me file explaning the features to other people.

In [37]:
data.to_csv(r"data\data.csv", index=False)

read_me = """# Cartola FC and Events Probabilites.

This dataset mix Cartola FC and game probabilities. Each row represents a player.

## Features

* position: Player's position. 
    0. Coach
    1. Goalkeeper
    2. Defender
    3. Fullback
    4. Midfielder
    5. Forward
* status: Player's status.
    0. Doubt
    1. Expected
* matches: Amount o matches that the player has played.
* mean: Player's points mean considering only games he has played.
* win: Probability of his team winning.
* lose: Probability of his team losing.
* draw: Probability of his team drawing.
* point: Amount of points the player scored on the match.

Author:

* [Matheus Couto](github.com/matheusccouto)

Sources:

* [github.com/henriquepgomide/caRtola](https://github.com/henriquepgomide/caRtola/tree/master/data)
* [football-data.co.uk/brazil.php](https://www.football-data.co.uk/brazil.php)

"""

with open(r"data\README.md", "w") as file:
    file.write(read_me)

## Model Construction
I will fit and evaluate some different types of modelling.

The goal is to choose the best predictor, but storage and speed also plays a role on the decision.

The models that I will fit are:

* Linear Regression
* Random Forrest
* XGBoost

### Split Dataset
Split the dataset into:

* `x`: features
* `y`: target

In [38]:
x = data.drop("points", axis=1)
x.head()

,position,status,matches,matches,mean,win,lose,draw
0,2,1,1.0,0.0,0.0,0.241808,0.384181,0.374011
1,4,1,1.0,0.0,0.0,0.384181,0.241808,0.374011
3,4,1,1.0,0.0,0.0,0.529870,0.134199,0.335931
6,4,1,1.0,0.0,0.0,0.241808,0.384181,0.374011
9,4,1,1.0,0.0,0.0,0.134199,0.529870,0.335931


In [39]:
y = data[["points"]]
y.head()

,points
0,0.5
1,7.5
3,0.9
6,0.8
9,6.5


### Define Pipelines
Some models needs preliminary steps. It is more modular if all models are inside pipelines. This way all individuals specifications can be easily integrated.

In [40]:
linear_pipe = sklearn.pipeline.Pipeline([
    ("scaler", sklearn.preprocessing.StandardScaler()),
    ("predictor", sklearn.linear_model.LinearRegression()),
])

forest_pipe = sklearn.pipeline.Pipeline([
    ("predictor", sklearn.ensemble.RandomForestRegressor()),
])
forest_grid = dict(
    predictor__n_estimators=[10],
    predictor__max_depth=[1, 2, 3, 4 ,5, 10, 50],
    predictor__min_samples_leaf=[10, 50, 100, 500],
    )

xgboost_pipe = sklearn.pipeline.Pipeline([
    ("predictor", sklearn.ensemble.RandomForestRegressor()),
])

In [41]:
%%time
forest_search = sklearn.model_selection.GridSearchCV(
    forest_pipe, param_grid=forest_grid, cv=5, scoring="neg_root_mean_squared_error"
)
forest_search.fit(x, y)
print(forest_search.best_estimator_)

Pipeline(steps=[('predictor',
                 RandomForestRegressor(max_depth=5, min_samples_leaf=10,
                                       n_estimators=10))])
Wall time: 43.5 s


In [42]:
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y)

rnd = sklearn.ensemble.RandomForestRegressor(n_estimators=10)
rnd.fit(x_train, y_train)

RandomForestRegressor(n_estimators=10)